In [1]:
# coding:utf-8
import argparse
#===============================================================
import os,sys,glob,re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import math
from scipy.stats import ttest_ind,linregress;
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import matthews_corrcoef
#from sklearn.metrics import recall_score
#from sklearn.metrics import precision_score
from sklearn.metrics import auc, mean_absolute_error, mean_squared_error, precision_recall_curve, \
r2_score,roc_auc_score, accuracy_score, log_loss
from sklearn.metrics import confusion_matrix,cohen_kappa_score
from sklearn.metrics import f1_score,confusion_matrix

In [2]:
h1_list = ['r2_score','pearson_r2','rmse','mse','mae','roc','prc','p-stat','Recall',
'Precision','f1','BA','accuracy', 'TN', 'FP', 'FN','TP','SP','SE','NPV','MCC']

In [3]:
def polyfit(x, y, degree):
	#coeffs = numpy.polyfit(x, y, degree) # Polynomial Coefficients
	correlation = numpy.corrcoef(x, y)[0,1]
	return correlation**2

def calc_class_roc_prc(y_true,y_pred,pos_label=1):
	#print('y_true =', y_true)
	#y_ = [y for y in y_true if y not in [0,1]]
	#print('y_ = ',y_)   ## -9223372036854775808, must be NaN
	x_roc=roc_auc_score(y_true,y_pred)
	if pos_label==0:x_roc = 1 - x_roc		
	x_precision, x_recall, x_thresholds = precision_recall_curve(y_true,y_pred)
	x_prc = auc(x_recall, x_precision)
	return {'roc':x_roc,'prc':x_prc}

def calc_class_other_stats_with_threshold(y_true, y_pred, threshold=0.5):
	ind0=np.where(y_true==0)[0];ind1=np.where(y_true==1)[0]
	y_pred0=y_pred[ind0];y_pred1=y_pred[ind1];
	p_0_1 = ttest_ind(y_pred0, y_pred1)[1]	 #"%.2e"%
	#print('p_stat = ', p_0_1)
	y_hard_pred = [1 if p > threshold else 0 for p in y_pred] # binary prediction
	#x_cohen_kappa = cohen_kappa_score(y_true, y_pred, weights='linear')	
	### not understand, can problem, Can't handle mix of binary and continuous
	x_cohen_kappa = cohen_kappa_score(y_true, y_hard_pred, weights='linear')
	## True and false values
	TN, FP, FN, TP = confusion_matrix(y_true, y_hard_pred).ravel()	
	#TP = sum((y_true == 1) & (y_hard_pred == 1))
	#TN = sum((y_true == 0) & (y_hard_pred == 0))
	#FN = sum((y_true == 1) & (y_hard_pred == 0))
	#FP = sum((y_true == 0) & (y_hard_pred == 1))
	# SE (Sensitivity), hit rate, recall, or true positive rate
	x_Recall = TP/(TP+FN)
	SE=x_Recall
	# Precision or positive predictive value
	x_Precision = TP/(TP+FP)
	#f1_score = (2*x_Precision*x_Recall)/ (x_Precision + x_Recall)
	f1 = f1_score(y_true, y_hard_pred)
	# CCR (Correct classification rate), BA (balanced accuracy)
	# Specificity or true negative rate
	SP = TN/(TN+FP) 
	x_BA = (SE + SP)/2
	x_accuracy = accuracy_score(y_true, y_hard_pred)
	# Negative predictive value
	NPV = TN/(TN+FN)
	x_MCC = matthews_corrcoef(y_true, y_hard_pred)
	other_stats = {'p_stat':p_0_1,'Recall':x_Recall,'Precision':x_Precision, 'f1':f1,'BA':x_BA,
	'accuracy':x_accuracy, 'TN':TN, 'FP':FP, 'FN':FN,'TP':TP,'SP':SP,'SE':SE,
	'NPV':NPV,'MCC':x_MCC,'cohen_kappa':x_cohen_kappa}  # 
	return other_stats

In [4]:
def calc_ef_threholds(y_true,y_pred,ef_threholds=[0.01,0.05,0.1]):
	# y_true,y_pred are np.array type
	df = pd.DataFrame()   ## merge  y_true,y_pred as df 's  two cols
	df['y_true'] = y_true;df['y_pred'] = y_pred
	n_actives = len(df[df['y_true']==1].index)
	n_total = len(df.index)
	random_rate = n_actives/ n_total
	df.sort_values(by='y_pred',ascending=False,inplace=True)
	EFs = {}
	for ef_threhold in ef_threholds:
		screen_range = int(np.ceil(n_total * ef_threhold))
		screen_rate = sum(df['y_true'][:screen_range]) / screen_range
		#print('n_actives,n_total = ',n_actives,n_total)
		print('random_rate,screen_range,screen_rate = ', random_rate,screen_range,screen_rate)
		EF = screen_rate / random_rate
		EF_name = 'EF_{}'.format(ef_threhold)
		#print('{} = {}'.format(EF_name,EF))
		EFs.update({EF_name:EF})
	return EFs

In [5]:
def list_stat_for_class(true_file,pred_file,pos_label,ef_threholds):
    t_df = pd.read_csv(true_file)
    p_df = pd.read_csv(pred_file)
    #t_values = t_df[label_col].values
    t_values = t_df.iloc[:,1].values.astype(int)
    if model == 'DMPNN': p_values = p_df.iloc[:,1].values
    else: p_values = p_df.iloc[:,0].values
    ind = np.where((t_values==0) | (t_values==1))[0]	 ### to remove those NaN label
    #print('ind = ',ind)
    y_true = t_values[ind]
    y_pred = p_values[ind]
    print('y_true[:10], y_pred[:10] =',y_true[:10], y_pred[:10])
    dic = {}
    dic.update(calc_class_roc_prc(y_true,y_pred,pos_label))
    dic.update(calc_ef_threholds(y_true,y_pred,ef_threholds))
    dic.update(calc_class_other_stats_with_threshold(y_true, y_pred,threshold))
    w_line= pred_file +',' + ','.join([str(dic[x]) for x in header])+'\n'
    f = open(eva_csv_name, 'a'); f.write(w_line); f.close()
    return 

In [6]:
efs=ef_threholds=[0.01,0.02,0.05]
pos_label = 1

# write header for eva_ensemble_with_rmSim.csv

In [44]:
fingerprint_list = ['MorganFP', 'RDkitFP', 'AtomPairFP', 'TorsionFP', 'AvalonFP', 
'EstateFP', 'MACCSFP', 'PharmacoErGFP', 'PharmacoPFP', 'PubChemFP', 'MHFP6', 'MAP4']
descriptor_list = ['Property', 'Constitution', 'Autocorr', 'Fragment', 'Charge', 
'Estate', 'MOE', 'Connectivity', 'Topology', 'Kappa', 'Path', 'Matrix', 'InfoContent']

num_folds = 5
#true_file = 'hide/rand_check/fold_0/train_full.csv'
b_list = ['train','val','test','Ext']   ## DMPNN use this

topNs = [140,130,120,100,90,80,70,60,50,40,30,25,20,15,10]
#thre=threshold=0.5
thresholds = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.85,0.9,0.95]

true_file_pat = 'rand_check_{feature}/fold_{i}/{b}_full.csv' 
true_file_e = 'Ext.csv'
#pred_file_pat = 'esb/800/p_train_Top70_fold2.csv'  # p_{b}_{feature}_fold{i}.csv' 
pred_file_pat = 'esb/800/p_{b}_Top{N}_fold{i}.csv'  # p_{b}_{feature}_fold{i}.csv' 
# include DMPNN

In [45]:
eva_csv_name = 'eva_ensemble_with_rmSim.csv'
dic = {}
y_true,y_pred =np.array([1,0]), np.array([0.9,0.1])
dic.update(calc_class_roc_prc(y_true,y_pred,pos_label))
dic.update(calc_ef_threholds(y_true,y_pred,ef_threholds))
dic.update(calc_class_other_stats_with_threshold(y_true, y_pred,threshold))
header=list(dic.keys()); #header=sorted(header)
tmp = [x for x in h1_list if x in header] + [x for x in header if x not in h1_list]
header = tmp
print('header=',header)
f = open(eva_csv_name, 'a');f.write('col_names,' + ','.join(header)+'\n');f.close()

random_rate,screen_range,screen_rate =  0.5 1 1.0
random_rate,screen_range,screen_rate =  0.5 1 1.0
random_rate,screen_range,screen_rate =  0.5 1 1.0
header= ['roc', 'prc', 'Recall', 'Precision', 'f1', 'BA', 'accuracy', 'TN', 'FP', 'FN', 'TP', 'SP', 'SE', 'NPV', 'MCC', 'EF_0.01', 'EF_0.02', 'EF_0.05', 'p_stat', 'cohen_kappa']


/tmp/ipykernel_1162945/448253304.py:19: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  p_0_1 = ttest_ind(y_pred0, y_pred1)[1]	 #"%.2e"%
/home/phzd/miniconda3/envs/chemprop1.4.0/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: divide by zero encountered in true_divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/home/phzd/miniconda3/envs/chemprop1.4.0/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/tmp/ipykernel_1162945/448253304.py:35: RuntimeWarning: invalid value encountered in long_scalars
  x_Precision = TP/(TP+FP)


# eva train,val,test,Ext for ensemble

In [46]:
model = '';feature = ''
threshold_values = []
for threshold in thresholds:
    for N in topNs:
        print(f'processing feature: {feature}; for ensemble model')
        for b in b_list: 
            for i in range(num_folds):
                if b=='Ext': true_file = 'Ext.csv'
                else: true_file = f'rand_{feature}/fold_{i}/{b}_full.csv'             
                pred_file = f'esb/800/p_{b}_Top{N}_fold{i}.csv'    # esb/800/p_{b}_{feature}_fold{i}.csv'  
                print('true_file,pred_file = ',true_file,pred_file)
                list_stat_for_class(true_file,pred_file,pos_label,ef_threholds)
                threshold_values.append(threshold)  
                ### write down the threshold which will be used in df create new columns

processing feature: ; for ensemble model
true_file,pred_file =  rand_/fold_0/train_full.csv esb/800/p_train_Top140_fold0.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 0 1 1 1] [0.9967457  0.9965024  0.9978409  0.9960239  0.05190892 0.99618345
 0.1221681  0.9955805  0.99876165 0.99555284]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
true_file,pred_file =  rand_/fold_1/train_full.csv esb/800/p_train_Top140_fold1.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 1 1 1 0] [0.02568613 0.9972505  0.99810046 0.99619776 0.99612373 0.99839324
 0.9986065  0.9973151  0.99717927 0.07371607]
random_rate,screen_range,screen_rate =  0.7771758757674251 28 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 56 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 139 1.0
true_file,pred_file =  rand_/fold_2/train_full.csv esb/800/p_train_To

/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)


true_file,pred_file =  rand_/fold_2/train_full.csv esb/800/p_train_Top130_fold2.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 0 1 1 1] [0.14398661 0.993588   0.9950734  0.99249506 0.99462044 0.9935644
 0.1693082  0.98773324 0.9929969  0.99826705]
random_rate,screen_range,screen_rate =  0.7804261466233298 28 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 56 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 139 1.0
true_file,pred_file =  rand_/fold_3/train_full.csv esb/800/p_train_Top130_fold3.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 0 1 1 1 0] [0.12881997 0.99512684 0.9983803  0.9971943  0.99649733 0.10005249
 0.99605894 0.9962081  0.9983638  0.18552782]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
true_file,pred_file =  rand_/fold_4/train_full.csv esb/800/p_train_Top130_fold4.csv
y_true[:10], y_pred[:10] = 

y_true[:10], y_pred[:10] = [0 1 1 1 1 0 1 1 1 0] [0.17925246 0.9959125  0.99820757 0.9973482  0.9966265  0.10619423
 0.9960251  0.99629176 0.9985587  0.199127  ]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
true_file,pred_file =  rand_/fold_4/train_full.csv esb/800/p_train_Top120_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 1 0 0 0] [0.99515533 0.9961211  0.99684733 0.9967757  0.06026238 0.998769
 0.99635834 0.098502   0.04245339 0.01447515]
random_rate,screen_range,screen_rate =  0.7833152762730228 28 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 56 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 139 1.0
true_file,pred_file =  rand_/fold_0/val_full.csv esb/800/p_val_Top120_fold0.csv
y_true[:10], y_pred[:10] = [1 0 1 1 1 1 1 1 1 1] [0.99433064 0.71487796 0.99245    0.9853593  0.9940831  0.99251294


/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid 

...

true_file,pred_file =  rand_/fold_3/train_full.csv esb/800/p_train_Top40_fold3.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 0 1 1 1 0] [0.6851533  0.9886622  0.98347926 0.9897859  0.9887158  0.53947926
 0.98980534 0.9863274  0.98945975 0.73785096]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
true_file,pred_file =  rand_/fold_4/train_full.csv esb/800/p_train_Top40_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 1 0 0 0] [0.9895167  0.98791915 0.98985684 0.989267   0.51115364 0.9874418
 0.9902946  0.470181   0.586177   0.45955878]
random_rate,screen_range,screen_rate =  0.7833152762730228 28 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 56 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 139 1.0
true_file,pred_file =  rand_/fold_0/val_full.csv esb/800/p_val_Top40_fold0.csv
y_true[:10], y_pred[:10] = [1 0 1 

true_file,pred_file =  rand_/fold_0/val_full.csv esb/800/p_val_Top30_fold0.csv
y_true[:10], y_pred[:10] = [1 0 1 1 1 1 1 1 1 1] [0.985911   0.9187701  0.9836474  0.97430444 0.98659766 0.9843336
 0.98623914 0.9855289  0.9403547  0.98670876]
random_rate,screen_range,screen_rate =  0.7658959537572254 4 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 7 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 18 0.9444444444444444
true_file,pred_file =  rand_/fold_1/val_full.csv esb/800/p_val_Top30_fold1.csv
y_true[:10], y_pred[:10] = [1 0 0 1 1 0 0 1 1 0] [0.9848232  0.5207923  0.9793921  0.9821454  0.98708165 0.5765458
 0.5231617  0.985851   0.986413   0.74190176]
random_rate,screen_range,screen_rate =  0.8063583815028902 4 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 7 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 18 1.0
true_file,pred_file =  rand_/fold_2/val_full.csv esb/800/p_val_Top30_fold2.csv
y_true[:10], y_pred[:10] = [1 1 0 

y_true[:10], y_pred[:10] = [1 0 1 0 1 1 0 1 0 1] [0.98728395 0.5068831  0.97884035 0.7470922  0.98636496 0.98553383
 0.7544003  0.9861085  0.97760266 0.98577917]
random_rate,screen_range,screen_rate =  0.7832369942196532 4 1.0
random_rate,screen_range,screen_rate =  0.7832369942196532 7 1.0
random_rate,screen_range,screen_rate =  0.7832369942196532 18 1.0
true_file,pred_file =  rand_/fold_0/test_full.csv esb/800/p_test_Top25_fold0.csv
y_true[:10], y_pred[:10] = [0 0 1 1 0 1 1 1 1 1] [0.489089   0.55889094 0.9848496  0.8708746  0.4817186  0.9859387
 0.98464924 0.98403823 0.8920259  0.98184156]
random_rate,screen_range,screen_rate =  0.7838616714697406 4 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 7 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 18 1.0
true_file,pred_file =  rand_/fold_1/test_full.csv esb/800/p_test_Top25_fold1.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 1 0 0] [0.9867002  0.98675597 0.9482202  0.9855838  0.52870834 0.5812894
 0.54703

random_rate,screen_range,screen_rate =  0.7694524495677233 4 1.0
random_rate,screen_range,screen_rate =  0.7694524495677233 7 1.0
random_rate,screen_range,screen_rate =  0.7694524495677233 18 0.9444444444444444
true_file,pred_file =  rand_/fold_4/test_full.csv esb/800/p_test_Top20_fold4.csv
y_true[:10], y_pred[:10] = [1 1 0 1 1 1 1 0 1 1] [0.9797642  0.98035103 0.61440814 0.9822749  0.51243466 0.97938997
 0.976829   0.6598383  0.97970325 0.98200744]
random_rate,screen_range,screen_rate =  0.7579250720461095 4 1.0
random_rate,screen_range,screen_rate =  0.7579250720461095 7 1.0
random_rate,screen_range,screen_rate =  0.7579250720461095 18 1.0
true_file,pred_file =  Ext.csv esb/800/p_Ext_Top20_fold0.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.98112065 0.9834963  0.9801139  0.9835941  0.98361534 0.9834487
 0.98015195 0.95488346 0.98343855 0.9833385 ]
random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 1.

true_file,pred_file =  Ext.csv esb/800/p_Ext_Top15_fold3.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.97654986 0.97951984 0.9747034  0.9796752  0.97972256 0.9796569
 0.9753456  0.9585436  0.97909236 0.97983426]
random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 22 1.0
true_file,pred_file =  Ext.csv esb/800/p_Ext_Top15_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.9794907  0.98229504 0.97888    0.9824859  0.9823093  0.98234576
 0.9781219  0.9578372  0.9820868  0.9820956 ]
random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 22 1.0
processing feature: ; for ensemble model
true_file,pred_file =  rand_/fold_0/train_full.csv esb/800/p_train_Top10_fold0.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1

# eva Ext_rm_sim

In [47]:
model = ''
ext_files=['Ext_rm_sim_0_6.csv','Ext_rm_sim_0_7.csv','Ext_rm_sim_0_8.csv','Ext_rm_sim_0_9.csv']
for threshold in thresholds:
    for N in topNs:
        for ext_file in ext_files: 
            for i in range(num_folds):
                ext_base = ext_file.replace('.csv','')
                true_file = ext_file      
                pred_file = f'esb/800/p_{ext_base}_Top{N}_fold{i}.csv'
                print('true_file,pred_file = ',true_file,pred_file)
                list_stat_for_class(true_file,pred_file,pos_label,ef_threholds) 
                threshold_values.append(threshold)  
                ### write down the threshold which will be used in df create new columns

true_file,pred_file =  Ext_rm_sim_0_6.csv esb/800/p_Ext_rm_sim_0_6_Top140_fold0.csv
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.977096   0.97455364 0.53330374 0.9721835  0.96254086 0.7714393
 0.7617511  0.555726   0.77229244 0.81659734]
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
true_file,pred_file =  Ext_rm_sim_0_6.csv esb/800/p_Ext_rm_sim_0_6_Top140_fold1.csv
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.98725986 0.9831634  0.7608447  0.9823837  0.9798859  0.84788346
 0.83810633 0.7371784  0.8653842  0.8402926 ]
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
true_file,pred_file =  Ext_rm_sim_0_6.csv esb/800/p_Ext_rm_sim_0_6_Top140_fold2.csv
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.98577857 0.9831396  0.5961018  0.9800591  0.9399422  0.8626139
 0.7

/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid 

random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 12 1.0
true_file,pred_file =  Ext_rm_sim_0_8.csv esb/800/p_Ext_rm_sim_0_8_Top130_fold3.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.9941273  0.9954716  0.9917042  0.92775553 0.9833413  0.99017227
 0.9819441  0.98397714 0.98248935 0.99308383]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 12 1.0
true_file,pred_file =  Ext_rm_sim_0_8.csv esb/800/p_Ext_rm_sim_0_8_Top130_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.9945695  0.99425936 0.9889675  0.9468395  0.9862649  0.9877079
 0.97412926 0.9815664  0.98562    0.99011046]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.6787330316

true_file,pred_file =  Ext_rm_sim_0_7.csv esb/800/p_Ext_rm_sim_0_7_Top100_fold0.csv
y_true[:10], y_pred[:10] = [0 0 0 0 0 1 1 1 1 1] [0.97984123 0.9889965  0.9901063  0.9755255  0.97588784 0.6470922
 0.98458683 0.984696   0.99146944 0.97651255]
random_rate,screen_range,screen_rate =  0.3968253968253968 1 1.0
random_rate,screen_range,screen_rate =  0.3968253968253968 2 1.0
random_rate,screen_range,screen_rate =  0.3968253968253968 4 0.5


/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid value encountered in long_scalars
  NPV = TN/(TN+FN)
/tmp/ipykernel_1162945/448253304.py:44: RuntimeWarning: invalid 

...

true_file,pred_file =  Ext_rm_sim_0_7.csv esb/800/p_Ext_rm_sim_0_7_Top40_fold3.csv
y_true[:10], y_pred[:10] = [0 0 0 0 0 1 1 1 1 1] [0.9631458  0.9696885  0.97605157 0.9717961  0.9750923  0.7231009
 0.9849726  0.97628367 0.984593   0.9833265 ]
random_rate,screen_range,screen_rate =  0.3968253968253968 1 1.0
random_rate,screen_range,screen_rate =  0.3968253968253968 2 1.0
random_rate,screen_range,screen_rate =  0.3968253968253968 4 1.0
true_file,pred_file =  Ext_rm_sim_0_7.csv esb/800/p_Ext_rm_sim_0_7_Top40_fold4.csv
y_true[:10], y_pred[:10] = [0 0 0 0 0 1 1 1 1 1] [0.95133907 0.9796087  0.97737175 0.97410935 0.9770439  0.7130927
 0.98537874 0.96236783 0.9838082  0.9774089 ]
random_rate,screen_range,screen_rate =  0.3968253968253968 1 1.0
random_rate,screen_range,screen_rate =  0.3968253968253968 2 1.0
random_rate,screen_range,screen_rate =  0.3968253968253968 4 1.0
true_file,pred_file =  Ext_rm_sim_0_8.csv esb/800/p_Ext_rm_sim_0_8_Top40_fold0.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0

y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.98642296 0.98481846 0.9823834  0.98384225 0.9860561  0.9872914
 0.9725044  0.98418933 0.9537855  0.9831934 ]
random_rate,screen_range,screen_rate =  0.7727272727272727 4 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 8 1.0
random_rate,screen_range,screen_rate =  0.7727272727272727 19 1.0
true_file,pred_file =  Ext_rm_sim_0_6.csv esb/800/p_Ext_rm_sim_0_6_Top25_fold0.csv
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.9658155  0.935135   0.65047973 0.8605909  0.86818165 0.71687824
 0.71579224 0.7757064  0.6917334  0.64837015]
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
true_file,pred_file =  Ext_rm_sim_0_6.csv esb/800/p_Ext_rm_sim_0_6_Top25_fold1.csv
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.97484154 0.9399739  0.74819076 0.91459525 0.875021   0.7331922
 0.7212787  0.88201314 0.73561734 0.7293507 ]
r

random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 12 1.0
true_file,pred_file =  Ext_rm_sim_0_8.csv esb/800/p_Ext_rm_sim_0_8_Top20_fold3.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.98258543 0.96570456 0.982383   0.8785177  0.95305437 0.94653964
 0.92619014 0.9595878  0.9491787  0.9815856 ]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 5 1.0
random_rate,screen_range,screen_rate =  0.6787330316742082 12 1.0
true_file,pred_file =  Ext_rm_sim_0_8.csv esb/800/p_Ext_rm_sim_0_8_Top20_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 0 0 1] [0.97827905 0.95767933 0.9785311  0.8545984  0.96181303 0.9391413
 0.9012849  0.95656615 0.94433224 0.9726105 ]
random_rate,screen_range,screen_rate =  0.6787330316742082 3 1.0
random_rate,screen_range,screen_rate =  0.678733031674

/tmp/ipykernel_1162945/448253304.py:35: RuntimeWarning: invalid value encountered in long_scalars
  x_Precision = TP/(TP+FP)
/tmp/ipykernel_1162945/448253304.py:35: RuntimeWarning: invalid value encountered in long_scalars
  x_Precision = TP/(TP+FP)
/tmp/ipykernel_1162945/448253304.py:35: RuntimeWarning: invalid value encountered in long_scalars
  x_Precision = TP/(TP+FP)


true_file,pred_file =  Ext_rm_sim_0_6.csv esb/800/p_Ext_rm_sim_0_6_Top10_fold3.csv
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.9152939  0.8446746  0.6061669  0.8338505  0.83970207 0.7695636
 0.67283726 0.83587    0.7550496  0.81321573]
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
true_file,pred_file =  Ext_rm_sim_0_6.csv esb/800/p_Ext_rm_sim_0_6_Top10_fold4.csv
y_true[:10], y_pred[:10] = [0 0 1 0 0 0 1 1 0 0] [0.92110336 0.8259198  0.65057224 0.80910873 0.7507537  0.59406394
 0.7406077  0.8785325  0.59459794 0.6204681 ]
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
random_rate,screen_range,screen_rate =  0.25 1 0.0
true_file,pred_file =  Ext_rm_sim_0_7.csv esb/800/p_Ext_rm_sim_0_7_Top10_fold0.csv
y_true[:10], y_pred[:10] = [0 0 0 0 0 1 1 1 1 1] [0.75087345 0.9738043  0.78665733 0.97655916 0.88165647 0.20300646
 0.990

In [48]:
print('eva_csv_name = ', eva_csv_name)
df = pd.read_csv(eva_csv_name)

eva_csv_name =  eva_ensemble_with_rmSim.csv


In [49]:
df

,col_names,roc,prc,Recall,Precision,f1,BA,accuracy,TN,FP,...,TP,SP,SE,NPV,MCC,EF_0.01,EF_0.02,EF_0.05,p_stat,cohen_kappa
0,esb/800/p_train_Top140_fold0.csv,0.999995,0.999999,1.000000,0.942559,0.970430,0.890547,0.952329,471,132,...,2166,0.781095,1.000000,1.000000,0.858037,1.278393,1.278393,1.278393,0.000000e+00,0.848077
1,esb/800/p_train_Top140_fold1.csv,0.999828,0.999955,1.000000,0.901173,0.948018,0.808752,0.914771,381,236,...,2152,0.617504,1.000000,1.000000,0.745974,1.286710,1.286710,1.286710,0.000000e+00,0.715047
2,esb/800/p_train_Top140_fold2.csv,0.999967,0.999991,1.000000,0.923899,0.960444,0.853618,0.935717,430,178,...,2161,0.707237,1.000000,1.000000,0.808341,1.281351,1.281351,1.281351,0.000000e+00,0.790383
3,esb/800/p_train_Top140_fold3.csv,0.999946,0.999985,1.000000,0.888433,0.940921,0.774461,0.901770,331,272,...,2166,0.548922,1.000000,1.000000,0.698341,1.278393,1.278393,1.278393,0.000000e+00,0.655625
4,esb/800/p_train_Top140_fold4.csv,0.999980,0.999994,1.000000,0.907531,0.951524,0.815833,0.920188,379,221,...,2169,0.631667,1.000000,1.000000,0.757138,1.276625,1.276625,1.276625,0.000000e+00,0.728752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6595,esb/800/p_Ext_rm_sim_0_9_Top10_fold0.csv,0.922288,0.977192,0.788927,0.978541,0.873563,0.865052,0.823529,80,5,...,228,0.941176,0.788927,0.567376,0.631329,1.294118,1.294118,1.294118,3.238959e-48,0.592364
6596,esb/800/p_Ext_rm_sim_0_9_Top10_fold1.csv,0.926766,0.976164,0.799308,0.966527,0.875000,0.852595,0.823529,77,8,...,231,0.905882,0.799308,0.570370,0.615317,1.294118,1.294118,1.294118,2.532609e-56,0.583954
6597,esb/800/p_Ext_rm_sim_0_9_Top10_fold2.csv,0.913780,0.974728,0.809689,0.962963,0.879699,0.851903,0.828877,76,9,...,234,0.894118,0.809689,0.580153,0.618262,1.294118,1.294118,1.294118,6.236427e-40,0.590936
6598,esb/800/p_Ext_rm_sim_0_9_Top10_fold3.csv,0.963647,0.989246,0.802768,0.978903,0.882129,0.871972,0.834225,80,5,...,232,0.941176,0.802768,0.583942,0.647090,1.294118,1.294118,1.294118,9.149504e-66,0.611838


In [50]:
# esb/800/p_train_Top140_fold0.csv , split column name into data_type, topN, fold
col_names = df['col_names'].values
m_list = [];data_list = [];topN_list = [];fold_list = []
for col in col_names:
    m,n,tmp = col.split('/')
    tmp = tmp.replace('.csv','').replace('p_Ext_rm_sim_0_','p_ExtRmSim0')
    _,dt,ft,fold = tmp.split('_')
    data_list.append(dt);topN_list.append(ft);fold_list.append(fold)
df['data'] = data_list
df['topN'] = topN_list
df['fold'] = fold_list

In [51]:
df['threshold'] = threshold_values

In [12]:
#df.columns.to_list()
#['col_names', 'roc', 'prc', 'Recall', 'Precision', 'f1', 'BA', 'accuracy', 'TN', 'FP', 'FN', 'TP', 'SP', 'SE',
# 'NPV', 'MCC', 'EF_0.01', 'EF_0.02', 'EF_0.05', 'p_stat', 'cohen_kappa']

In [52]:
def group_thres_topN_for_esn_performance_sort(xeva = 'roc'):
    group_test = test_df.groupby(['threshold','topN']).mean()[xeva]
    group_test_std = test_df.groupby(['threshold','topN']).std()[xeva]
    group_ext = ext_df.groupby(['threshold','topN']).mean()[xeva]
    group_ext_std = ext_df.groupby(['threshold','topN']).std()[xeva]

    group_train = train_df.groupby(['threshold','topN']).mean()[xeva]
    group_train_std = train_df.groupby(['threshold','topN']).std()[xeva]
    group_val = val_df.groupby(['threshold','topN']).mean()[xeva]
    group_val_std = val_df.groupby(['threshold','topN']).std()[xeva]

    dfl = [pd.DataFrame() for i in [6,7,8,9]] 
    groupl = [group_val for i in [6,7,8,9]] 
    group_stdl = [group_val_std for i in [6,7,8,9]] 
    tmp_list = []
    for i,sim in enumerate([6,7,8,9]):
        #dfl[0]=df[df['data']=='train']
        dfl[i]=df[df['data']==f'ExtRmSim0{sim}']
        groupl[i] = dfl[i].groupby(['threshold','topN']).mean()[xeva]
        group_stdl[i] = dfl[i].groupby(['threshold','topN']).std()[xeva]
        tmp_list.append(groupl[i]);tmp_list.append(group_stdl[i])
    grouped_model_feat = pd.concat([group_train,group_train_std,group_val,group_val_std,
                group_test,group_test_std,group_ext,group_ext_std]+tmp_list,axis=1)
    grouped_model_feat.columns = [f'train_{xeva}',f'train_{xeva}_std',f'val_{xeva}',f'val_{xeva}_std',
              f'test_{xeva}',f'test_{xeva}_std', f'ext_{xeva}',f'ext_{xeva}_std'] + \
            ['rmSim_0.6','rmSim_0.6_std','rmSim_0.7','rmSim_0.7_std',
             'rmSim_0.8','rmSim_0.8_std', 'rmSim_0.9','rmSim_0.9_std']
    grouped_model_feat.to_csv(f'grouped_esb_metric/grouped_esb_{xeva}.csv')
    grouped_model_feat = pd.read_csv(f'grouped_esb_metric/grouped_esb_{xeva}.csv')
    grouped_sort=grouped_model_feat.sort_values(by=[f'ext_{xeva}',f'test_{xeva}',],ascending=False)
    grouped_sort.to_csv(f'grouped_esb_metric/grouped_esb_sort_{xeva}.csv',index=False)
    return grouped_sort

In [53]:
test_df=df[df['data']=='test']
ext_df=df[df['data']=='Ext']
train_df=df[df['data']=='train']
val_df=df[df['data']=='train']

In [54]:
if not os.path.exists(f'grouped_esb_metric'):os.mkdir(f'grouped_esb_metric')
grouped = []
if not os.path.exists('grouped_metric'):os.mkdir('grouped_metric')
for x_metric in ['roc', 'prc', 'Recall', 'Precision', 'f1', 'BA', 'accuracy', 'TN', 'FP', 'FN', 'TP', 'SP', 'SE',
'NPV', 'MCC', 'EF_0.01', 'EF_0.02', 'EF_0.05', 'p_stat', 'cohen_kappa']:
    grouped.append(group_thres_topN_for_esn_performance_sort(x_metric))